In [1]:
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import prim
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ema_workbench import MultiprocessingEvaluator, ema_logging
import pickle

In [2]:
model, _ = get_model_for_problem_formulation(2)

In [ ]:
from ema_workbench import Samplers

ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios = 1000 #In assignment 5 is at first 1000 used and later on for the dimensional stacking 10000. 
n_policies = 8 #Not sure if we have to go for 8 again here like in the earlier exploratory modeling

use_pickle1 = False
if use_pickle1:
    with open('data/PRIM_results1.pickle','rb') as filehandler:
        results = pickle.load(filehandler)
else:
    with MultiprocessingEvaluator(model,n_processes=10) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, n_policies, uncertainty_sampling=Samplers.LHS)
    
    with open('data/PRIM_results1.pickle',"wb") as filehandler:
        pickle.dump(results,filehandler)

[MainProcess/INFO] pool started with 10 workers
[MainProcess/INFO] performing 1000 scenarios * 8 policies * 1 model(s) = 8000 experiments
  8%|███▎                                   | 667/8000 [00:59<11:36, 10.52it/s]

In [ ]:
experiments, outcomes = results

In [ ]:
cleaned_experiments = experiments.drop(labels=[l.name for l in model.levers], axis=1)
##!!! Waarom worden de policies er hier uit gehaald

In [ ]:
import numpy as np

data = outcomes['Expected Number of Deaths']
y = data < np.percentile(data, 95) #Not sure which percentile we need to use
###!!! Ook jan kwakkel heeft bij zijn analyse een y met true/false gemaakt
###!!! Maar waarom ziet de prim grafiek er als een bijna rechte lijn uit

In [ ]:
#threshold (float) – the density threshold that a box has to meet
#Not sure which tresholds we need to use?
prim_alg = prim.Prim(cleaned_experiments,y, threshold=0.8, peel_alpha=0.05) 
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
box1.inspect()
box1.inspect(style='graph')
plt.show()

In [ ]:
from ema_workbench.analysis import dimensional_stacking

In [ ]:
dimensional_stacking.create_pivot_plot(cleaned_experiments, y)
plt.show()